In [ ]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark
from pyspark.sql.types import IntegerType, TimestampType, DoubleType
from pyspark.sql.functions import col, collect_set, to_timestamp, sum as spark_sum
import json
from tqdm import tqdm
from datetime import datetime
import random
from typing import List, Dict, Optional
import pytest

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/diploma/scripts")
import process_data

In [ ]:
# PARAMETERS
HEAD_DIRECTORY = '/content/drive/MyDrive/Colab Notebooks/diploma/'
path_raw_actions = HEAD_DIRECTORY+'data/needed_beh_logs'
path_raw_items = HEAD_DIRECTORY+'data/data_set_items'
path_processed_actions = HEAD_DIRECTORY+'data/sasrec_format/actions'
path_processed_items = HEAD_DIRECTORY+'data/sasrec_format/items'
action = 'conversion'
metadata_original_names = ['group_ids']
facets_original_names = ['How to get it', 'Primary Brand', 'Primary Pet Type']
metadata_final_names = ['Category']
facets_final_names = ['Delivery', 'Brand', 'Pet']

# Тесты для датасета с действиями юзеров в формате SASRec

In [ ]:
# Создаем SparkSession
spark = SparkSession.builder \
    .appName("PetCo_indirect_tests") \
    .getOrCreate()

NameError: name 'SparkSession' is not defined

In [ ]:
# Читаем файлы
data_raw_actions = spark.read.parquet(path_raw_actions)
data_processed_actions = spark.read.parquet(path_processed_actions)

In [ ]:
@pytest.mark.parametrize("df", [(data_processed_actions)])
def test_actions_structure(df: pyspark.sql.DataFrame) -> None:
    """
    Проверяет структуру датасета: названия колонок и типы данных.

    Args:
        df (pyspark.sql.DataFrame): Предобработанный датасет с действиями юзеров в формате SASRec.

    Returns:
        None: Функция выполняет проверки, но ничего не возвращает.
    """

    expected_columns = {"user_id", "item_id", "datetime", "weight"}
    actual_columns = set(df.columns)
    assert actual_columns == expected_columns, f"Ошибка: Ожидались колонки {expected_columns}, но получены {actual_columns}"

    # Получаем схему DataFrame
    schema = dict(df.dtypes)

    # Проверка типов данных
    assert schema["user_id"] == "bigint", "Ошибка: 'user_id' должен быть большим целым числом"
    assert schema["item_id"] == "int", "Ошибка: 'item_id' должен быть целым числом"
    assert schema["datetime"] == "timestamp", "Ошибка: 'datetime' должен быть datetime"
    assert schema["weight"] == "int", "Ошибка: 'weight' должен быть числом"

    print("✅ В датасете правильные колонки и у них правильные типы данных")


@pytest.mark.parametrize("df", [(data_processed_actions)])
def test_missing_values_actions(df: pyspark.sql.DataFrame) -> None:
    """
    Проверяет наличие пропущенных значений в датасете.

    Args:
        df (pyspark.sql.DataFrame): Предобработанный датасет с действиями юзеров в формате SASRec.

    Returns:
        None: Функция выполняет проверки, но ничего не возвращает.
    """
    # spark, process_data = start_script()

    missing_values = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

    missing_dict = missing_values.collect()[0].asDict()
    total_missing = sum(missing_dict.values())

    assert total_missing == 0, f"Ошибка: Найдены пропущенные значения\n{missing_dict}"

    print("✅ В датасете нет пропущенных значений")


@pytest.mark.parametrize("df_final, df_original, action", [(data_processed_actions, data_raw_actions, action)])
def test_number_users(df_final: pyspark.sql.DataFrame, df_original: pyspark.sql.DataFrame, action: str) -> None:
    """
    Проверяет, совпадает ли количество уникальных пользователей в обработанном и исходном датасете.

    Args:
        df_final (pyspark.sql.DataFrame): Предобработанный датасет с действиями юзеров в формате SASRec.
        df_original (pyspark.sql.DataFrame): Исходный датасет с действиями юзеров (needed_beh_logs).
        action (str): Название действия (например, 'click', 'purchase').

    Returns:
        None: Функция выполняет проверки, но ничего не возвращает.
    """

    n_users_original = (
        df_original.filter(col("action") == action)
        .select("customer_user_id")
        .distinct()
        .count()
    )

    n_users_final = df_final.select("user_id").distinct().count()

    assert n_users_final == n_users_original, "Ошибка: неправильное число юзеров"

    print("✅ В датасете правильное число юзеров, которые сделали действие")


@pytest.mark.parametrize("df_final, df_original, action", [(data_processed_actions, data_raw_actions, action)])
def test_users_actions(df_final: pyspark.sql.DataFrame, df_original: pyspark.sql.DataFrame, action: str, n_users: int = 3) -> None:
    """
    Проверяет, что для случайных пользователей корректно указаны товары и время действий.

    Args:
        df_final (pyspark.sql.DataFrame): Финальный датасет с действиями юзеров в формате SASRec.
        df_original (pyspark.sql.DataFrame): Исходный датасет с действиями юзеров (needed_beh_logs)
        action (str): Название действия юзеров.
        n_users (int, optional): Количество пользователей для проверки. По умолчанию 10.

    Returns:
        None: Функция выполняет проверки, но ничего не возвращает.
    """
    users = df_final.select("user_id").distinct().rdd.flatMap(lambda x: x).takeSample(False, n_users)

    for user in tqdm(users):
        check_user_actions(user, df_final, df_original, action)

    print("\n✅ Для всех юзеров правильно указаны товары и время действий")


def check_user_actions(user_id: int, df_final: pyspark.sql.DataFrame, df_original: pyspark.sql.DataFrame, action: str) -> None:
    """
    Проверяет, совпадают ли товары и время действий пользователя в обработанном и исходном датасете.

    Args:
        user_id (int): Идентификатор пользователя.
        df_final (pyspark.sql.DataFrame): Предобработанный датасет с действиями юзеров в формате SASRec.
        df_original (pyspark.sql.DataFrame): Исходный датасет с действиями юзеров (needed_beh_logs)
        action (str): Название действия.

    Returns:
        None: Функция выполняет проверки, но ничего не возвращает.
    """
    assert get_user_items_from_final(user_id, df_final) == get_user_items_from_original(user_id, df_original, action), \
        f"Ошибка: неправильно указаны набор товаров и время действий для юзера {user_id}"


def get_user_items_from_final(user_id: int, df: pyspark.sql.DataFrame) -> set[tuple[int, datetime]]:
    """
    Получает множество пар (item_id, datetime) для пользователя из обработанного датасета.

    Args:
        user_id (int): Идентификатор пользователя.
        df (pyspark.sql.DataFrame): Финальный датасет с действиями юзеров

    Returns:
        set[tuple[int, datetime]]: Множество пар (item_id, datetime).
    """
    df_filtered = df.filter(col("user_id") == user_id).select("item_id", "datetime")

    return set(df_filtered.rdd.map(lambda row: (row["item_id"], row["datetime"])).collect())


def get_user_items_from_original(user_id: int, df: pyspark.sql.DataFrame, action: str) -> set[tuple[int, datetime]]:
    """
    Получает множество пар (customer_id, timestamp) для пользователя из исходного датасета.

    Args:
        user_id (int): Идентификатор пользователя.
        df (pyspark.sql.DataFrame): Исходный датасет с действиями юзеров (needed_beh_logs).
        action (str): Название действия.

    Returns:
        set[tuple[int, datetime]]: Множество пар (customer_id, timestamp).
    """
    df_filtered = (
        df.filter(col("action") == action)
        .filter(col("customer_user_id") == str(user_id))
        .select("customer_id", "timestamp")
    )

    df_filtered = df_filtered.withColumn("timestamp", to_timestamp(col("timestamp")))

    return set(df_filtered.rdd.map(lambda row: (int(row["customer_id"]), row["timestamp"].replace(microsecond=0))).collect())

# Тесты для датасета с признаками товаров в формате SASRec

In [ ]:
# Читаем файлы
data_raw_items = spark.read.parquet(path_raw_items)
data_processed_items = spark.read.parquet(path_processed_items)

In [ ]:
@pytest.mark.parametrize("df", [(data_processed_items)])
def test_items_structure(df: pyspark.sql.DataFrame) -> None:
    """
    Проверяет структуру датасета с товарами.

    Args:
        df (pyspark.sql.DataFrame): Предобработанный датасет с признаками товаров в формате SASRec.

    Returns:
        None: Функция выполняет проверки, но ничего не возвращает.
    """
    expected_columns = {"id", "value", "feature"}
    actual_columns = set(df.columns)
    assert actual_columns == expected_columns, f"Ошибка: Ожидались колонки {expected_columns}, но получены {actual_columns}"

    schema = df.schema
    assert schema["id"].dataType.simpleString() == "int", "Ошибка: 'id' должен быть целым числом"
    assert schema["value"].dataType.simpleString() == "string", "Ошибка: 'value' должен быть строкой"
    assert schema["feature"].dataType.simpleString() == "string", "Ошибка: 'feature' должен быть строкой"

    print("✅ В датасете правильные колонки и у них правильные типы данных")


@pytest.mark.parametrize("df", [(data_processed_items)])
def test_missing_values_items(df: pyspark.sql.DataFrame) -> None:
    """
    Проверяет наличие пропущенных значений в датасете.

    Args:
        df (pyspark.sql.DataFrame): Предобработанный датасет с признаками товаров в формате SASRec.

    Returns:
        None: Функция выполняет проверки, но ничего не возвращает.
    """
    missing_values = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
    missing_dict = missing_values.collect()[0].asDict()
    total_missing = sum(missing_dict.values())
    assert total_missing == 0, f"Ошибка: Найдены пропущенные значения\n{missing_dict}"

    print("✅ В датасете нет пропущенных значений")


@pytest.mark.parametrize("df_final, df_original", [(data_processed_items, data_raw_items)])
def test_number_items(df_final: pyspark.sql.DataFrame, df_original: pyspark.sql.DataFrame) -> None:
    """
    Проверяет соответствие числа товаров в обработанном и исходном датасетах.

    Args:
        df_final (pyspark.sql.DataFrame): Предобработанный датасет с признаками товаров в формате SASRec.
        df_original (pyspark.sql.DataFrame): Исходный каталог (data_set_items).

    Returns:
        None: Функция выполняет проверки, но ничего не возвращает.
    """
    n_items_original = df_original.select("customer_id").distinct().count()
    n_items_final = df_final.select("id").distinct().count()
    assert n_items_final == n_items_original, "Ошибка: число товаров не совпадает с каталогом"

    print("✅ Число товаров в датасете совпадает с каталогом")


@pytest.mark.parametrize("df_final, df_original, metadata_original_names, facets_original_names, metadata_final_names, facets_final_names", \
 [(data_processed_items, data_raw_items,  metadata_original_names, facets_original_names, metadata_final_names, facets_final_names)])
def test_items_features(
    df_final: pyspark.sql.DataFrame,
    df_original: pyspark.sql.DataFrame,
    metadata_original_names: List[str],
    facets_original_names: List[str],
    metadata_final_names: List[str],
    facets_final_names: List[str],
    n_items: int = 3
) -> None:
    """
    Проверяет соответствие признаков товаров в обработанном и исходном датасетах.

    Args:
        df_final (pyspark.sql.DataFrame): Предобработанный датасет с признаками товаров в формате SASRec.
        df_original (pyspark.sql.DataFrame): Исходный каталог (data_set_items).
        metadata_original_names (List[str]): Список названий метаданных в исходном датасете.
        facets_original_names (List[str]): Список названий фасетов в исходном датасете.
        metadata_final_names (List[str]): Список названий метаданных в предобработанном датасете.
        facets_final_names (List[str]): Список названий фасетов в предобработанном датасете.
        n_items (int, optional): Количество случайных товаров для проверки. По умолчанию 10.

    Returns:
        None: Функция выполняет проверки, но ничего не возвращает.
    """
    items = df_final.select("id").distinct().rdd.flatMap(lambda x: x).takeSample(False, n_items)
    for item in tqdm(items):
        check_item_features(item, df_final, df_original, metadata_original_names, facets_original_names, metadata_final_names, facets_final_names)

    print("\n✅ Признаки товаров в датасете совпадают с каталогом")


def check_item_features(
    item_id: int,
    df_final: pyspark.sql.DataFrame,
    df_original: pyspark.sql.DataFrame,
    metadata: List[str],
    facets: List[str],
    metadata_final_names: List[str],
    facets_final_names: List[str]
) -> None:
    """
    Проверяет корректность признаков конкретного товара.

    Args:
        item_id (int): Идентификатор товара.
        df_final (pyspark.sql.DataFrame): Предобработанный датасет с признаками товаров в формате SASRec.
        df_original (pyspark.sql.DataFrame): Исходный каталог (data_set_items).
        metadata (List[str]): Список метаданных в исходном датасете.
        facets (List[str]): Список фасетов в исходном датасете.
        metadata_final_names (List[str]): Список названий метаданных в предобработанном датасете.
        facets_final_names (List[str]): Список названий фасетов в предобработанном датасете.

    Returns:
        None: Функция выполняет проверки, но ничего не возвращает.
    """
    features_original_names = metadata + facets
    features_final_names = metadata_final_names + facets_final_names
    features_names_changes = {features_original_names[i]: features_final_names[i] for i in range(len(features_original_names))}

    item_features_final = item_features_from_final(item_id, df_final)
    item_features_original = {features_names_changes[key]: value for key, value in item_features_from_original(item_id, df_original, metadata, facets).items()}

    assert set(item_features_final).issubset(item_features_original), 'Ошибка: Финальный набор категорий содержит лишнюю категорию'
    assert len(item_features_final) > 0, f'Ошибка: У товара {item_id} нет никаких признаков'


def item_features_from_original(
    item_id: int,
    df: pyspark.sql.DataFrame,
    metadata: List[str],
    facets: Optional[List[str]] = None
) -> Dict[str, List[str]]:
    """
    Извлекает признаки товара из исходного датасета.

    Args:
        item_id (int): Идентификатор товара.
        df (pyspark.sql.DataFrame): Исходный каталог (data_set_items).
        metadata (List[str]): Список метаданных.
        facets (List[str], optional): Список фасетов. По умолчанию None.

    Returns:
        Dict[str, List[str]]: Словарь с признаками товара.
    """
    features_out = {}
    df = df.filter(col('customer_id') == item_id).select('metadata_json')
    metadata_in = json.loads(df.first()[0])

    for metadata_type in metadata:
        features_out[metadata_type] = metadata_in[metadata_type]

    if facets:
        facets_in = metadata_in.get('facets', {})
        for facet in facets:
            if facet in facets_in:
                features_out[facet] = facets_in[facet]

    return features_out


def item_features_from_final(item_id: int, dataset: pyspark.sql.DataFrame) -> Dict[str, List[str]]:
    """
    Извлекает признаки товара из обработанного датасета.

    Args:
        item_id (int): Идентификатор товара.
        dataset (pyspark.sql.DataFrame): Предобработанный датасет с признаками товаров в формате SASRec.

    Returns:
        Dict[str, List[str]]: Словарь, где ключ — это feature, а значения — список уникальных value.
    """
    filtered_data = dataset.filter(dataset.id == item_id)
    result = (
        filtered_data
        .groupBy("feature")
        .agg(collect_set("value").alias("values"))
        .rdd
        .map(lambda row: (row["feature"], row["values"]))
        .collectAsMap()
    )

    return result

# Своместные тесты на оба датасета в формате SASRec


In [ ]:
@pytest.mark.parametrize("df_actions, df_items", [(data_processed_actions, data_processed_items)])
def test_items_with_actions_in_catalog(df_actions: pyspark.sql.DataFrame, df_items: pyspark.sql.DataFrame) -> None:
    """
    Проверяет, что почти все товары из действий пользователей присутствуют в каталоге товаров.

    Args:
        df_actions (DataFrame): Предобработанный датасет с действиями пользователей в формате SASRec.
        df_items (DataFrame): Предобработанный датасет с признаками товаров в формате SASRec.

    Returns:
        None

    Raises:
        AssertionError: Если в финальном датасете товаров слишком мало товаров
                        или информация отсутствует более чем о 0.1% товаров.
    """
    all_items_from_actions = set(df_actions.select("item_id").distinct().rdd.map(lambda row: row["item_id"]).collect())
    all_items_from_catalog = set(df_items.select("id").distinct().rdd.map(lambda row: row["id"]).collect())

    assert len(all_items_from_actions) <= len(all_items_from_catalog), 'Ошибка: В финальном датасете товаров слишком мало товаров'
    assert len(all_items_from_actions - all_items_from_catalog) < 0.001 * len(all_items_from_actions), 'Ошибка: О многих товарах нет информации в финальном датасете товаров'

    print("✅ В финальном датасете товаров есть 99.9% всех товаров, с которыми взаимодействовали юзеры")